<a href="https://colab.research.google.com/github/codar030/the-turing-way/blob/master/Cinematica_ARM_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# Lista de encoders basada en la información proporcionada
encoders_list = [
    [
        {'nombre': 'Actuador 4 Yaw', 'tipo': 'rotacional', 'valor': 13, 'hole_id': '1'},
        {'nombre': 'Cilindro 6 Pitch', 'tipo': 'rotacional', 'valor': 38},
        {'nombre': 'Cilindro 7 Prismatic', 'tipo': 'cuerda', 'valor': 100},
        {'nombre': 'Cilindro 8 Pitch', 'tipo': 'rotacional', 'valor': 15},
        {'nombre': 'Actuador 1 Yaw', 'tipo': 'rotacional', 'valor': 57},
        {'nombre': 'Cilindro 1 Pitch', 'tipo': 'rotacional', 'valor': 14},
        {'nombre': 'Cilindro 2 Prismatic', 'tipo': 'cuerda', 'valor': 10},
        {'nombre': 'Actuador 2 Roll', 'tipo': 'rotacional', 'valor': 182},
        {'nombre': 'Actuador 3 Yaw', 'tipo': 'rotacional', 'valor': 35},
        {'nombre': 'Cilindro 3 Pitch', 'tipo': 'rotacional', 'valor': 95},
        {'nombre': 'Cilindro 4 Prismatic', 'tipo': 'cuerda', 'valor': 10}
    ],

    [
        {'nombre': 'Actuador 4 Yaw', 'tipo': 'rotacional', 'valor': 25, 'hole_id': '2'},
        {'nombre': 'Cilindro 6 Pitch', 'tipo': 'rotacional', 'valor': 40},
        {'nombre': 'Cilindro 7 Prismatic', 'tipo': 'cuerda', 'valor': 300},
        {'nombre': 'Cilindro 8 Pitch', 'tipo': 'rotacional', 'valor': 15},
        {'nombre': 'Actuador 1 Yaw', 'tipo': 'rotacional', 'valor': 55},
        {'nombre': 'Cilindro 1 Pitch', 'tipo': 'rotacional', 'valor': 40},
        {'nombre': 'Cilindro 2 Prismatic', 'tipo': 'cuerda', 'valor': 400},
        {'nombre': 'Actuador 2 Roll', 'tipo': 'rotacional', 'valor': 75},
        {'nombre': 'Actuador 3 Yaw', 'tipo': 'rotacional', 'valor': 35},
        {'nombre': 'Cilindro 3 Pitch', 'tipo': 'rotacional', 'valor': 20},
        {'nombre': 'Cilindro 4 Prismatic', 'tipo': 'cuerda', 'valor': 200}
    ],

    [
        {'nombre': 'Actuador 4 Yaw', 'tipo': 'rotacional', 'valor': 39, 'hole_id': '49'}, #1 Giro Brazo Normet
        {'nombre': 'Cilindro 6 Pitch', 'tipo': 'rotacional', 'valor': 14}, #2 Levante Brazo Normet
        {'nombre': 'Cilindro 7 Prismatic', 'tipo': 'cuerda', 'valor': 1}, #4 Extensión Brazo Normet
        {'nombre': 'Cilindro 8 Pitch', 'tipo': 'rotacional', 'valor': 16}, #3 Primador
        {'nombre': 'Actuador 1 Yaw', 'tipo': 'rotacional', 'valor': 48}, #5 Giro Brazo Delantero
        {'nombre': 'Cilindro 1 Pitch', 'tipo': 'rotacional', 'valor': 17}, #6 Levante Brazo Chico
        {'nombre': 'Cilindro 2 Prismatic', 'tipo': 'cuerda', 'valor': 0.1}, #7 Extensión Brazo delantero
        {'nombre': 'Actuador 2 Roll', 'tipo': 'rotacional', 'valor': 180}, #11 Rotación Cabezal
        {'nombre': 'Actuador 3 Yaw', 'tipo': 'rotacional', 'valor': 66}, #9 Giro Cabezal
        {'nombre': 'Cilindro 3 Pitch', 'tipo': 'rotacional', 'valor': 22},#8 Levante cabezal
        {'nombre': 'Cilindro 4 Prismatic', 'tipo': 'cuerda', 'valor': 10} # Extension cabezal
    ],


]

In [47]:
# Longitudes de los segmentos
longitudes = {
    'Cilindro 1': 0,     # Longitud de Cilindro 1
    'Cilindro 2': 500,   # Longitud de Cilindro 2
    'Cilindro 3': 0,     # Longitud de Cilindro 3
    'Cilindro 4': 300,   # Longitud de Cilindro 4
    'Cilindro 6': 0,     # Longitud de Cilindro 6
    'Cilindro 7': 350,   # Longitud de Cilindro 7
    'Cilindro 8': 0      # Longitud de Cilindro 8
}


In [48]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def dh_transform(a, alpha, d, theta):
    """Crea una matriz de transformación homogénea usando parámetros DH."""
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

def calcular_posiciones_dh(encoders_list, longitudes):
    """Calcula las posiciones finales del brazo para múltiples listas de encoders usando DH."""
    posiciones_finales = []
    todas_las_posiciones = []
    hole_ids = []

    for encoders in encoders_list:
        posiciones = [np.array([0.0, 0.0, 0.0, 1.0])]
        T_total = np.eye(4)  # Matriz de identidad para iniciar
        hole_id = encoders[0]['hole_id']  # Asumimos que el hole_id es el mismo para todo el conjunto

        for encoder in encoders:
            nombre, tipo, valor = encoder['nombre'], encoder['tipo'], encoder['valor']

            if tipo == 'rotacional':
                theta = np.deg2rad(valor)  # Convertir de grados a radianes
                a = 0
                alpha = 0
                d = 0
                if 'yaw' in nombre.lower():
                    alpha = np.pi / 2
                elif 'pitch' in nombre.lower():
                    alpha = 0
                elif 'roll' in nombre.lower():
                    alpha = -np.pi / 2
            elif tipo == 'cuerda':
                theta = 0
                a = 0
                alpha = 0
                d = valor

            # Si el encoder corresponde a un cilindro, agregar la longitud del cilindro como un desplazamiento a lo largo del eje X
            if 'Cilindro' in nombre:
                num_cilindro = nombre.split()[1]  # Extraer el número de cilindro
                a = longitudes.get(f'Cilindro {num_cilindro}', 0)

            # Crear la matriz de transformación homogénea DH
            T = dh_transform(a, alpha, d, theta)
            T_total = T_total @ T

            # Agregar la posición calculada
            posiciones.append(T_total @ np.array([0.0, 0.0, 0.0, 1.0]))

        posiciones = np.array(posiciones)
        posiciones_finales.append(posiciones[-1, :3])  # Solo las posiciones finales
        todas_las_posiciones.append(posiciones[:, :3])  # Todas las posiciones
        hole_ids.append(hole_id)

    return np.array(posiciones_finales), todas_las_posiciones, hole_ids


# Calcular posiciones finales y todas las posiciones para múltiples listas de encoders
posiciones_finales, todas_las_posiciones, hole_ids = calcular_posiciones_dh(encoders_list, longitudes)

# Datos para el último conjunto de encoders
posiciones = todas_las_posiciones[-1]

# Crear la figura inicial con subplots
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.4, 0.4],  # Ajustar el ancho de las columnas para dar más espacio
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter'}]],
    subplot_titles=('Brazo Robótico en 3D', 'Posiciones Finales del Brazo en 2D'),
    horizontal_spacing=0.05  # Aumentar el espacio horizontal entre los gráficos
)

# Dimensiones ajustadas del área de trabajo (frente de carga) para estar dentro del alcance del brazo
ancho_frente = 1500  # mm (eje Y)
alto_frente = 1000  # mm (eje X)
profundidad_frente = 1000  # mm (grosor en eje Z)

# Centro del frente de carga en el eje Y
centro_frente_y = ancho_frente / 2

# Ajustar las posiciones del brazo robótico para los nuevos ejes
x = posiciones[:, 0]  # Eje Z (longitud del brazo) se visualiza en el eje X
y = posiciones[:, 1]  # Eje Y (movimiento lateral) se visualiza en el eje Y
z = posiciones[:, 2]  # Eje X (altura) se visualiza en el eje Z

#x = posiciones[:, 2]  # Eje Z (longitud del brazo) se visualiza en el eje X
#y = posiciones[:, 1]  # Eje Y (movimiento lateral) se visualiza en el eje Y
#z = posiciones[:, 0]  # Eje X (altura) se visualiza en el eje Z

# Agregar la proyección 3D del brazo robótico
fig.add_trace(
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='blue', width=2),
        name='Brazo Robótico'
    ),
    row=1, col=1
)

# Crear el área de trabajo (frente de carga) como una superficie vertical perpendicular al eje Z
y_surface = np.linspace(-centro_frente_y, centro_frente_y, 50)
x_surface = np.linspace(0, alto_frente, 50)
y_surface, x_surface = np.meshgrid(y_surface, x_surface)
z_surface = np.zeros_like(y_surface) + profundidad_frente

fig.add_trace(
    go.Surface(
        x=x_surface, y=y_surface, z=z_surface,
        opacity=0.2,
        colorscale='Greys',
        showscale=False,
        name='Frente de carguío'
    ),
    row=1, col=1
)

# Agregar un pequeño vector tridimensional en el origen para el eje X (longitud del brazo)
fig.add_trace(
    go.Scatter3d(
        x=[0, 100], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        line=dict(color='green', width=3),
        marker=dict(size=3, color='green'),
        name='Vector X'
    ),
    row=1, col=1
)

# Agregar un pequeño vector tridimensional en el origen para el eje Y (movimiento lateral)
fig.add_trace(
    go.Scatter3d(
        x=[0, 0], y=[0, 100], z=[0, 0],
        mode='lines+markers',
        line=dict(color='blue', width=3),
        marker=dict(size=3, color='blue'),
        name='Vector Y'
    ),
    row=1, col=1
)

# Agregar un pequeño vector tridimensional en el origen para el eje Z (altura)
fig.add_trace(
    go.Scatter3d(
        x=[0, 0], y=[0, 0], z=[0, 100],
        mode='lines+markers',
        line=dict(color='red', width=3),
        marker=dict(size=3, color='red'),
        name='Vector Z'
    ),
    row=1, col=1
)

# Agregar los puntos finales en la gráfica 3D con etiquetas
for i, (x_final, y_final, z_final, hole_id) in enumerate(zip(posiciones_finales[:, 0], posiciones_finales[:, 1], posiciones_finales[:, 2], hole_ids)):
    fig.add_trace(
        go.Scatter3d(
            x=[x_final], y=[y_final], z=[z_final],
            mode='markers+text',
            marker=dict(size=8, color='purple'),
            text=[hole_id],
            textposition='top center',
            name=f'Posición Final {hole_id}'
        ),
        row=1, col=1
    )

# Agregar la posición final en la gráfica 2D con etiquetas
for i, (x_final, y_final, hole_id) in enumerate(zip(posiciones_finales[:, 0], posiciones_finales[:, 1], hole_ids)):
    fig.add_trace(
        go.Scatter(
            x=[x_final], y=[y_final],
            mode='markers+text',
            marker=dict(size=10, color='purple'),
            text=[hole_id],
            textposition='top center',
            name=f'Posición Final {hole_id}'
        ),
        row=1, col=2
    )

# Configurar el layout del gráfico y ajustar el tamaño
fig.update_layout(
    title='Posiciones del Brazo Robótico en 3D y 2D',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
        aspectmode='manual',
        aspectratio=dict(x=1.5, y=1.5, z=1)  # Ajustar la proporción de los ejes
    ),
    xaxis2=dict(title='Posición X'),
    yaxis2=dict(title='Posición Y'),
    showlegend=True,
    width=1600,  # Ajustar el ancho del gráfico
    height=900   # Ajustar la altura del gráfico
)

# Mostrar el gráfico
fig.show()


In [151]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

def dh_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

params = [
    (0, 0, 0, np.pi/2),    # DOF_1 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_2 (ROT:PITCH)
    (0, 0, 3600, np.pi/2),  # DOF_3 (CUE:PRIS)
    (0, 0, 794, 0),        # Eslabón entre DOF_4 y DOF_5
    (0, 0, 0, -np.pi/2),   # DOF_5 (ROT:YAW)
    (0, 0, 0, np.pi/2),    # DOF_6 (ROT:PITCH)
    (0, 0, 1952.7, 0),     # DOF_7 (CUE:PRIS)
    (0, 0, 0, 0),          # DOF_8 (ROT:ROLL - eje X)

    (0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH) - Nuevo
    (0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW) - Nuevo

    #(0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW)
    #(0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH)
    (0, 0, 1141, 0)        # DOF_11 (CUE:PRIS)
]

altura_base = 1500

def forward_kinematics(DOF):
    theta1 = np.radians(DOF["theta1"])
    theta2 = np.radians(DOF["theta2"])
    theta4 = np.radians(DOF["theta4"])
    theta5 = np.radians(DOF["theta5"])
    theta6 = np.radians(DOF["theta6"])
    theta8 = np.radians(DOF["theta8"])
    theta9 = np.radians(DOF["theta9"])
    theta10 = np.radians(DOF["theta10"])

    d3_increment = DOF["d3_increment"]
    d7_increment = DOF["d7_increment"]
    d11_increment = DOF["d11_increment"]

    T = np.eye(4)
    T[2, 3] = altura_base
    positions = [T[:3, 3]]
    reference_vectors = []  # Almacena los vectores de referencia

    thetas = [theta1, theta2, 0, theta4, 0, theta5, theta6, theta8, theta9, theta10]
    prismatics = [d3_increment, d7_increment, d11_increment]
    prismatics_idx = 0

    for i, (theta, d, a, alpha) in enumerate(params):
        if i in [2, 6, 10]:  # Índices de los DOF prismáticos
            a += prismatics[prismatics_idx]  # Aumentar el desplazamiento d con el incremento prismático
            prismatics_idx += 1
        if i < len(thetas) and thetas[i] != 0:
            theta = thetas[i]
        # Asegurar que la rotación en el DOF 8 se aplique correctamente
        if i == 7:  # DOF 8 (ROLL)
            A = np.array([
                [1, 0, 0, 0],
                [0, np.cos(theta), -np.sin(theta), 0],
                [0, np.sin(theta), np.cos(theta), 0],
                [0, 0, 0, 1]
            ])
        else:
            A = dh_matrix(theta, d, a, alpha)
        T = np.dot(T, A)
        positions.append(T[:3, 3])

    # Crear vector de referencia en la punta del brazo
    ref_vector_start = positions[-1]
    ref_vector_end = np.dot(T, np.array([0, 500, 0, 1]))[:3]  # Vector de 500 unidades
    reference_vectors.append((ref_vector_start, ref_vector_end))

    return np.array(positions), T, reference_vectors  # Devolver las posiciones, la matriz de transformación final y los vectores de referencia

def plot_robot(DOF):
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
    positions, T_final, reference_vectors = forward_kinematics(DOF)
    x = positions[:, 0]
    y = positions[:, 1]
    z = positions[:, 2]

    trace = go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                         marker=dict(size=5, color='red'), line=dict(color='blue', width=2))
    fig.add_trace(trace)

    # Añadir el vector de referencia en la punta del brazo
    if reference_vectors:
        ref_start, ref_end = reference_vectors[0]
        fig.add_trace(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                                   y=[ref_start[1], ref_end[1]],
                                   z=[ref_start[2], ref_end[2]],
                                   mode='lines',
                                   marker=dict(size=5, color='green'), line=dict(color='green', width=2)))

    fig.update_layout(scene=dict(xaxis=dict(title='X', range=[0, 8000]),
                                 yaxis=dict(title='Y', range=[-7000, 7000]),
                                 zaxis=dict(title='Z', range=[0, 7000])),
                      title='Robotics ARM UGiT')

    fig.show()

# Datos de ejemplo
json_data = '''
{
    "theta1": 0,
    "theta2": 30,
    "theta4": 30,
    "theta5": 30,
    "theta6": 30,
    "theta8": 0,
    "theta9": 45,
    "theta10": 90,
    "d3_increment": 0,
    "d7_increment": 0,
    "d11_increment": 0
}
'''

DOF = json.loads(json_data)

# Generar la figura
plot_robot(DOF)


In [51]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

def dh_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

params = [
    (0, 0, 0, np.pi/2),    # DOF_1 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_2 (ROT:PITCH)
    (0, 0, 3600, np.pi/2),  # DOF_3 (CUE:PRIS)
    (0, 0, 794, 0),        # Eslabón entre DOF_4 y DOF_5
    (0, 0, 0, -np.pi/2),   # DOF_5 (ROT:YAW)
    (0, 0, 0, np.pi/2),    # DOF_6 (ROT:PITCH)
    (0, 0, 1952.7, 0),     # DOF_7 (CUE:PRIS)
    (0, 0, 0, 0),          # DOF_8 (ROT:ROLL en el eje X)
    (0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH)
    (0, 0, 1141, 0)        # DOF_11 (CUE:PRIS)
]

altura_base = 1500

def forward_kinematics(DOF):
    theta1 = np.radians(DOF["theta1"])
    theta2 = np.radians(DOF["theta2"])
    theta4 = np.radians(DOF["theta4"])
    theta5 = np.radians(DOF["theta5"])
    theta6 = np.radians(DOF["theta6"])
    theta8 = np.radians(DOF["theta8"])
    theta9 = np.radians(DOF["theta9"])
    theta10 = np.radians(DOF["theta10"])

    d3_increment = DOF["d3_increment"]
    d7_increment = DOF["d7_increment"]
    d11_increment = DOF["d11_increment"]

    T = np.eye(4)
    T[2, 3] = altura_base
    positions = [T[:3, 3]]
    reference_vectors = []  # Almacena los vectores de referencia

    thetas = [theta1, theta2, 0, theta4, 0, theta5, theta6, theta8, theta9, theta10]
    prismatics = [d3_increment, d7_increment, d11_increment]
    prismatics_idx = 0

    for i, (theta, d, a, alpha) in enumerate(params):
        if i in [2, 6, 10]:  # Índices de los DOF prismáticos
            d += prismatics[prismatics_idx]  # Aumentar el desplazamiento d con el incremento prismático
            prismatics_idx += 1
        if i < len(thetas) and thetas[i] != 0:
            theta = thetas[i]
        # Asegurar que la rotación en el DOF 8 se aplique correctamente
        if i == 7:  # DOF 8 (ROLL)
            A = np.array([
                [1, 0, 0, 0],
                [0, np.cos(theta), -np.sin(theta), 0],
                [0, np.sin(theta), np.cos(theta), 0],
                [0, 0, 0, 1]
            ])
        else:
            A = dh_matrix(theta, d, a, alpha)
        T = np.dot(T, A)
        positions.append(T[:3, 3])

    # Crear vector de referencia en la punta del brazo
    ref_vector_start = positions[-1]
    ref_vector_end = np.dot(T, np.array([0, 100, 0, 1]))[:3]  # Vector de 100 unidades en Y desde la punta
    reference_vectors.append((ref_vector_start, ref_vector_end))

    return np.array(positions), T, reference_vectors  # Devolver las posiciones, la matriz de transformación final y los vectores de referencia


In [52]:
# Datos de ejemplo en formato JSON
json_data = '''
[
    {"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 30, "theta8": 0, "theta9": 0, "theta10": 0, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0},
    {"id_tiro": 2, "theta1": 10, "theta2": 20, "theta4": 0, "theta5": 0, "theta6": 10, "theta8": 60, "theta9": 0, "theta10": 10, "d3_increment": 0, "d7_increment": 30, "d11_increment": 100},
    {"id_tiro": 3, "theta1": -5, "theta2": 10, "theta4": 0, "theta5": 10, "theta6": 0, "theta8": 0, "theta9": 0, "theta10": 4, "d3_increment":0 , "d7_increment":0 , "d11_increment":0 }
]
'''

frames_data = json.loads(json_data)


In [53]:
def create_animation(frames_data):
    frames = []
    all_final_positions = []  # Para almacenar todas las posiciones finales progresivamente

    for i, DOF in enumerate(frames_data):
        positions, _, reference_vectors = forward_kinematics(DOF)
        x = positions[:, 0]
        y = positions[:, 1]
        z = positions[:, 2]

        data = [go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                             marker=dict(size=5, color='red'),
                             line=dict(color='blue', width=2))]

        if reference_vectors:
            ref_start, ref_end = reference_vectors[0]
            data.append(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                                     y=[ref_start[1], ref_end[1]],
                                     z=[ref_start[2], ref_end[2]],
                                     mode='lines',
                                     marker=dict(size=5, color='green'),
                                     line=dict(color='green', width=2)))

        # Añadir punto final con ID del tiro para la posición actual
        final_position = positions[-1]
        all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

        # Incluir todas las posiciones finales acumuladas
        for pos in all_final_positions:
            data.append(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                                     mode='markers+text', text=[str(pos[3])],  # Línea modificada
                                     marker=dict(size=5, color='orange')))

        frames.append(go.Frame(data=data, name=str(DOF["id_tiro"])))

    return frames




# Crear la animación
frames = create_animation(frames_data)


In [54]:
# Crear la figura inicial
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
positions, _, reference_vectors = forward_kinematics(frames_data[0])
x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

trace = go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                     marker=dict(size=5, color='red'), line=dict(color='blue', width=2))
fig.add_trace(trace)


# Añadir el vector de referencia inicial
if reference_vectors:
    ref_start, ref_end = reference_vectors[0]
    fig.add_trace(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                               y=[ref_start[1], ref_end[1]],
                               z=[ref_start[2], ref_end[2]],
                               mode='lines',
                               marker=dict(size=5, color='green'),
                               line=dict(color='green', width=2)))

# Añadir los puntos finales de todas las posiciones a la figura inicial
all_final_positions = []

for DOF in frames_data:
    positions, _, _ = forward_kinematics(DOF)
    final_position = positions[-1]
    all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

for pos in all_final_positions:
    fig.add_trace(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                               mode='markers+text', text=[str(pos[3])],
                               marker=dict(size=5, color='orange')))

# Añadir los frames a la figura
fig.frames = frames

# Configurar la animación
animation_settings = dict(
    frame=dict(duration=500, redraw=True),
    fromcurrent=True,
    transition=dict(duration=0)
)

# Añadir los botones de reproducción de la animación
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, animation_settings]),
                               dict(label='Pause',
                                    method='animate',
                                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])],
    scene=dict(xaxis=dict(title='X', showticklabels=False, showgrid=True, showline=True),
               yaxis=dict(title='Y', showticklabels=False, showgrid=True, showline=True),
               zaxis=dict(title='Z', showticklabels=False, showgrid=True, showline=True)),
    title='Brazo Robótico Animado'
)

# Mostrar la figura
fig.show()


In [56]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

def dh_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

params = [
    (0, 0, 0, np.pi/2),    # DOF_1 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_2 (ROT:PITCH)
    (0, 0, 3600, np.pi/2),  # DOF_3 (CUE:PRIS)
    (0, 0, 794, 0),        # Eslabón entre DOF_4 y DOF_5
    (0, 0, 0, -np.pi/2),   # DOF_5 (ROT:YAW)
    (0, 0, 0, np.pi/2),    # DOF_6 (ROT:PITCH)
    (0, 0, 1952.7, 0),     # DOF_7 (CUE:PRIS)
    (0, 0, 0, 0),          # DOF_8 (ROT:ROLL en el eje X)
    (0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH)
    (0, 0, 1141, 0)        # DOF_11 (CUE:PRIS)
]

altura_base = 1500

def forward_kinematics(DOF):
    theta1 = np.radians(DOF["theta1"])
    theta2 = np.radians(DOF["theta2"])
    theta4 = np.radians(DOF["theta4"])
    theta5 = np.radians(DOF["theta5"])
    theta6 = np.radians(DOF["theta6"])
    theta8 = np.radians(DOF["theta8"])
    theta9 = np.radians(DOF["theta9"])
    theta10 = np.radians(DOF["theta10"])

    d3_increment = DOF["d3_increment"]
    d7_increment = DOF["d7_increment"]
    d11_increment = DOF["d11_increment"]

    T = np.eye(4)
    T[2, 3] = altura_base
    positions = [T[:3, 3]]
    reference_vectors = []  # Almacena los vectores de referencia

    thetas = [theta1, theta2, 0, theta4, 0, theta5, theta6, theta8, theta9, theta10]
    prismatics = [d3_increment, d7_increment, d11_increment]
    prismatics_idx = 0

    for i, (theta, d, a, alpha) in enumerate(params):
        if i in [2, 6, 10]:  # Índices de los DOF prismáticos
            d += prismatics[prismatics_idx]  # Aumentar el desplazamiento d con el incremento prismático
            prismatics_idx += 1
        if i < len(thetas) and thetas[i] != 0:
            theta = thetas[i]
        # Asegurar que la rotación en el DOF 8 se aplique correctamente
        if i == 7:  # DOF 8 (ROLL)
            A = np.array([
                [1, 0, 0, 0],
                [0, np.cos(theta), -np.sin(theta), 0],
                [0, np.sin(theta), np.cos(theta), 0],
                [0, 0, 0, 1]
            ])
        else:
            A = dh_matrix(theta, d, a, alpha)
        T = np.dot(T, A)
        positions.append(T[:3, 3])

    # Crear vector de referencia en la punta del brazo
    ref_vector_start = positions[-1]
    ref_vector_end = np.dot(T, np.array([0, 100, 0, 1]))[:3]  # Vector de 100 unidades en Y desde la punta
    reference_vectors.append((ref_vector_start, ref_vector_end))

    return np.array(positions), T, reference_vectors  # Devolver las posiciones, la matriz de transformación final y los vectores de referencia


# Datos de ejemplo en formato JSON
json_data = '''
[
    {"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 30, "theta8": 0, "theta9": 0, "theta10": 0, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0},
    {"id_tiro": 2, "theta1": 10, "theta2": 20, "theta4": 0, "theta5": 0, "theta6": 10, "theta8": 60, "theta9": 0, "theta10": 10, "d3_increment": 0, "d7_increment": 30, "d11_increment": 100},
    {"id_tiro": 3, "theta1": -5, "theta2": 10, "theta4": 0, "theta5": 10, "theta6": 0, "theta8": 0, "theta9": 0, "theta10": 4, "d3_increment":0 , "d7_increment":0 , "d11_increment":0 }
]
'''

frames_data = json.loads(json_data)

def create_animation(frames_data):
    frames = []
    all_final_positions = []  # Para almacenar todas las posiciones finales progresivamente

    for i, DOF in enumerate(frames_data):
        positions, _, reference_vectors = forward_kinematics(DOF)
        x = positions[:, 0]
        y = positions[:, 1]
        z = positions[:, 2]

        data = [go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                             marker=dict(size=5, color='red'),
                             line=dict(color='blue', width=2))]

        if reference_vectors:
            ref_start, ref_end = reference_vectors[0]
            data.append(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                                     y=[ref_start[1], ref_end[1]],
                                     z=[ref_start[2], ref_end[2]],
                                     mode='lines',
                                     marker=dict(size=5, color='green'),
                                     line=dict(color='green', width=2)))

        # Añadir punto final con ID del tiro para la posición actual
        final_position = positions[-1]
        all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

        # Incluir todas las posiciones finales acumuladas
        for pos in all_final_positions:
            data.append(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                                     mode='markers+text', text=[str(pos[3])],  # Línea modificada
                                     marker=dict(size=5, color='orange')))

        frames.append(go.Frame(data=data, name=str(DOF["id_tiro"])))

    return frames

# Crear la animación
frames = create_animation(frames_data)

# Crear la figura inicial
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
positions, _, reference_vectors = forward_kinematics(frames_data[0])
x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

trace = go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                     marker=dict(size=5, color='red'), line=dict(color='blue', width=2))
fig.add_trace(trace)


# Añadir el vector de referencia inicial
if reference_vectors:
    ref_start, ref_end = reference_vectors[0]
    fig.add_trace(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                               y=[ref_start[1], ref_end[1]],
                               z=[ref_start[2], ref_end[2]],
                               mode='lines',
                               marker=dict(size=5, color='green'),
                               line=dict(color='green', width=2)))

# Añadir los puntos finales de todas las posiciones a la figura inicial
all_final_positions = []

for DOF in frames_data:
    positions, _, _ = forward_kinematics(DOF)
    final_position = positions[-1]
    all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

for pos in all_final_positions:
    fig.add_trace(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                               mode='markers+text', text=[str(pos[3])],
                               marker=dict(size=5, color='orange')))

# Añadir los frames a la figura
fig.frames = frames

# Agregar visualización de la frente de carguío (suponiendo que es un plano vertical en X = 7300)
frente_x = 7300  # Posición en el eje X del frente de carguío
frente_y = np.linspace(-2500, 2500, 50)  # Rango en el eje Y
frente_z = np.linspace(0, 5000, 50)  # Rango en el eje Z
frente_y, frente_z = np.meshgrid(frente_y, frente_z)

# Crear la curvatura en la parte superior del frente de carguío
curvatura_superior = 1000 * np.exp(-((frente_y / 2500) ** 2))

# Aplicar la curvatura al frente de carguío
frente_z = np.where(frente_z > 4000, frente_z + curvatura_superior, frente_z)

fig.add_trace(go.Surface(x=frente_x * np.ones_like(frente_y), y=frente_y, z=frente_z,
                         colorscale='Viridis', opacity=0.5))

# Configurar la animación
animation_settings = dict(
    frame=dict(duration=500, redraw=True),
    fromcurrent=True,
    transition=dict(duration=0)
)

# Añadir los botones de reproducción de la animación
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, animation_settings]),
                               dict(label='Pause',
                                    method='animate',
                                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])],
    scene=dict(xaxis=dict(title='X', showticklabels=False, showgrid=True, showline=True),
               yaxis=dict(title='Y', showticklabels=False, showgrid=True, showline=True),
               zaxis=dict(title='Z', showticklabels=False, showgrid=True, showline=True)),
    title='Brazo Robótico Animado'
)

# Mostrar la figura
fig.show()


In [57]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

def dh_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

params = [
    (0, 0, 0, np.pi/2),    # DOF_1 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_2 (ROT:PITCH)
    (0, 0, 3600, np.pi/2),  # DOF_3 (CUE:PRIS)
    (0, 0, 794, 0),        # Eslabón entre DOF_4 y DOF_5
    (0, 0, 0, -np.pi/2),   # DOF_5 (ROT:YAW)
    (0, 0, 0, np.pi/2),    # DOF_6 (ROT:PITCH)
    (0, 0, 1952.7, 0),     # DOF_7 (CUE:PRIS)
    (0, 0, 0, 0),          # DOF_8 (ROT:ROLL en el eje X)
    (0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH)
    (0, 0, 1141, 0)        # DOF_11 (CUE:PRIS)
]

altura_base = 1500

def forward_kinematics(DOF):
    theta1 = np.radians(DOF["theta1"])
    theta2 = np.radians(DOF["theta2"])
    theta4 = np.radians(DOF["theta4"])
    theta5 = np.radians(DOF["theta5"])
    theta6 = np.radians(DOF["theta6"])
    theta8 = np.radians(DOF["theta8"])
    theta9 = np.radians(DOF["theta9"])
    theta10 = np.radians(DOF["theta10"])

    d3_increment = DOF["d3_increment"]
    d7_increment = DOF["d7_increment"]
    d11_increment = DOF["d11_increment"]

    T = np.eye(4)
    T[2, 3] = altura_base
    positions = [T[:3, 3]]
    reference_vectors = []  # Almacena los vectores de referencia

    thetas = [theta1, theta2, 0, theta4, 0, theta5, theta6, theta8, theta9, theta10]
    prismatics = [d3_increment, d7_increment, d11_increment]
    prismatics_idx = 0

    for i, (theta, d, a, alpha) in enumerate(params):
        if i in [2, 6, 10]:  # Índices de los DOF prismáticos
            d += prismatics[prismatics_idx]  # Aumentar el desplazamiento d con el incremento prismático
            prismatics_idx += 1
        if i < len(thetas) and thetas[i] != 0:
            theta = thetas[i]
        # Asegurar que la rotación en el DOF 8 se aplique correctamente
        if i == 7:  # DOF 8 (ROLL)
            A = np.array([
                [1, 0, 0, 0],
                [0, np.cos(theta), -np.sin(theta), 0],
                [0, np.sin(theta), np.cos(theta), 0],
                [0, 0, 0, 1]
            ])
        else:
            A = dh_matrix(theta, d, a, alpha)
        T = np.dot(T, A)
        positions.append(T[:3, 3])

    # Crear vector de referencia en la punta del brazo
    ref_vector_start = positions[-1]
    ref_vector_end = np.dot(T, np.array([0, 100, 0, 1]))[:3]  # Vector de 100 unidades en Y desde la punta
    reference_vectors.append((ref_vector_start, ref_vector_end))

    return np.array(positions), T, reference_vectors  # Devolver las posiciones, la matriz de transformación final y los vectores de referencia


# Datos de ejemplo en formato JSON
json_data = '''
[
    {"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 30, "theta8": 0, "theta9": 0, "theta10": 0, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0},
    {"id_tiro": 2, "theta1": 10, "theta2": 20, "theta4": 0, "theta5": 0, "theta6": 10, "theta8": 60, "theta9": 0, "theta10": 10, "d3_increment": 0, "d7_increment": 30, "d11_increment": 100},
    {"id_tiro": 3, "theta1": -5, "theta2": 10, "theta4": 0, "theta5": 10, "theta6": 0, "theta8": 0, "theta9": 0, "theta10": 4, "d3_increment":0 , "d7_increment":0 , "d11_increment":0 }
]
'''

frames_data = json.loads(json_data)

def create_animation(frames_data):
    frames = []
    all_final_positions = []  # Para almacenar todas las posiciones finales progresivamente

    for i, DOF in enumerate(frames_data):
        positions, _, reference_vectors = forward_kinematics(DOF)
        x = positions[:, 0]
        y = positions[:, 1]
        z = positions[:, 2]

        data = [go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                             marker=dict(size=5, color='red'),
                             line=dict(color='blue', width=2))]

        if reference_vectors:
            ref_start, ref_end = reference_vectors[0]
            data.append(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                                     y=[ref_start[1], ref_end[1]],
                                     z=[ref_start[2], ref_end[2]],
                                     mode='lines',
                                     marker=dict(size=5, color='green'),
                                     line=dict(color='green', width=2)))

        # Añadir punto final con ID del tiro para la posición actual
        final_position = positions[-1]
        all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

        # Incluir todas las posiciones finales acumuladas
        for pos in all_final_positions:
            data.append(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                                     mode='markers+text', text=[str(pos[3])],  # Línea modificada
                                     marker=dict(size=5, color='orange')))

        frames.append(go.Frame(data=data, name=str(DOF["id_tiro"])))

    return frames

# Crear la animación
frames = create_animation(frames_data)

# Crear la figura inicial
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
positions, _, reference_vectors = forward_kinematics(frames_data[0])
x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

trace = go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                     marker=dict(size=5, color='red'), line=dict(color='blue', width=2))
fig.add_trace(trace)


# Añadir el vector de referencia inicial
if reference_vectors:
    ref_start, ref_end = reference_vectors[0]
    fig.add_trace(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                               y=[ref_start[1], ref_end[1]],
                               z=[ref_start[2], ref_end[2]],
                               mode='lines',
                               marker=dict(size=5, color='green'),
                               line=dict(color='green', width=2)))

# Añadir los puntos finales de todas las posiciones a la figura inicial
all_final_positions = []

for DOF in frames_data:
    positions, _, _ = forward_kinematics(DOF)
    final_position = positions[-1]
    all_final_positions.append((final_position[0], final_position[1], final_position[2], DOF["id_tiro"]))

for pos in all_final_positions:
    fig.add_trace(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                               mode='markers+text', text=[str(pos[3])],
                               marker=dict(size=5, color='orange')))

# Añadir los frames a la figura
fig.frames = frames

# Agregar visualización de la frente de carguío (suponiendo que es un plano vertical en X = 7300)
frente_x = 7300  # Posición en el eje X del frente de carguío
frente_y = np.linspace(-2500, 2500, 50)  # Rango en el eje Y
frente_z = np.linspace(0, 5000, 50)  # Rango en el eje Z
frente_y, frente_z = np.meshgrid(frente_y, frente_z)

# Crear la curvatura en la parte superior del frente de carguío
curvatura_superior = 1000 * np.exp(-((frente_y / 2500) ** 2))

# Aplicar la curvatura al frente de carguío
frente_z = np.where(frente_z > 4000, frente_z + curvatura_superior, frente_z)

# Definir un mapa de colores personalizado que se asemeje a la roca
colorscale_roca = [
    [0.0, 'rgb(102, 51, 0)'],
    [0.5, 'rgb(153, 76, 0)'],
    [1.0, 'rgb(204, 102, 0)']
]

fig.add_trace(go.Surface(x=frente_x * np.ones_like(frente_y), y=frente_y, z=frente_z,
                         colorscale=colorscale_roca, opacity=0.4))

# Configurar la animación
animation_settings = dict(
    frame=dict(duration=500, redraw=True),
    fromcurrent=True,
    transition=dict(duration=0)
)

# Añadir los botones de reproducción de la animación
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, animation_settings]),
                               dict(label='Pause',
                                    method='animate',
                                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])],
    scene=dict(xaxis=dict(title='X', showticklabels=False, showgrid=True, showline=True),
               yaxis=dict(title='Y', showticklabels=False, showgrid=True, showline=True),
               zaxis=dict(title='Z', showticklabels=False, showgrid=True, showline=True)),
    title='Brazo Robótico Animado'
)

# Mostrar la figura
fig.show()


In [110]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json

def dh_matrix(theta, d, a, alpha):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

params = [
    (0, 0, 0, np.pi/2),    # DOF_1 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_2 (ROT:PITCH)
    (0, 0, 3600, np.pi/2),  # DOF_3 (CUE:PRIS)
    (0, 0, 794, 0),        # Eslabón entre DOF_4 y DOF_5
    (0, 0, 0, -np.pi/2),   # DOF_5 (ROT:YAW)
    (0, 0, 0, np.pi/2),    # DOF_6 (ROT:PITCH)
    (0, 0, 1952.7, 0),     # DOF_7 (CUE:PRIS)
    (0, 0, 0, 0),          # DOF_8 (ROT:ROLL en el eje X)
    (0, 0, 0, np.pi/2),    # DOF_9 (ROT:YAW)
    (0, 0, 0, -np.pi/2),   # DOF_10 (ROT:PITCH)
    (0, 0, 1141, 0)        # DOF_11 (CUE:PRIS)
]

altura_base = 2200

def forward_kinematics(DOF):
    theta1 = np.radians(DOF["theta1"])
    theta2 = np.radians(DOF["theta2"])
    theta4 = np.radians(DOF["theta4"])
    theta5 = np.radians(DOF["theta5"])
    theta6 = np.radians(DOF["theta6"])
    theta8 = np.radians(DOF["theta8"])
    theta9 = np.radians(DOF["theta9"])
    theta10 = np.radians(DOF["theta10"])

    d3_increment = DOF["d3_increment"]
    d7_increment = DOF["d7_increment"]
    d11_increment = DOF["d11_increment"]

    T = np.eye(4)
    T[2, 3] = altura_base
    positions = [T[:3, 3]]
    reference_vectors = []  # Almacena los vectores de referencia

    thetas = [theta1, theta2, 0, theta4, 0, theta5, theta6, theta8, theta9, theta10]
    prismatics = [d3_increment, d7_increment, d11_increment]
    prismatics_idx = 0

    for i, (theta, d, a, alpha) in enumerate(params):
        if i in [2, 6, 10]:  # Índices de los DOF prismáticos
            d += prismatics[prismatics_idx]  # Aumentar el desplazamiento d con el incremento prismático
            prismatics_idx += 1
        if i < len(thetas) and thetas[i] != 0:
            theta = thetas[i]
        # Asegurar que la rotación en el DOF 8 se aplique correctamente
        if i == 7:  # DOF 8 (ROLL)
            A = np.array([
                [1, 0, 0, 0],
                [0, np.cos(theta), -np.sin(theta), 0],
                [0, np.sin(theta), np.cos(theta), 0],
                [0, 0, 0, 1]
            ])
        else:
            A = dh_matrix(theta, d, a, alpha)
        T = np.dot(T, A)
        positions.append(T[:3, 3])

    # Crear vector de referencia en la punta del brazo
    ref_vector_start = positions[-1]
    ref_vector_end = np.dot(T, np.array([0, 100, 0, 1]))[:3]  # Vector de 100 unidades en Y desde la punta
    reference_vectors.append((ref_vector_start, ref_vector_end))

    return np.array(positions), T, reference_vectors  # Devolver las posiciones, la matriz de transformación final y los vectores de referencia


# Datos de ejemplo en formato JSON
json_data = '''
[
    {"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 30, "theta8": 0, "theta9": 45, "theta10": 0, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0},
    {"id_tiro": 2, "theta1": 10, "theta2": 20, "theta4": 0, "theta5": 0, "theta6": 10, "theta8": 60, "theta9": 0, "theta10": 10, "d3_increment": 0, "d7_increment": 30, "d11_increment": 100},
    {"id_tiro": 3, "theta1": -5, "theta2": 10, "theta4": 0, "theta5": 10, "theta6": 0, "theta8": 0, "theta9": 0, "theta10": 4, "d3_increment":0 , "d7_increment":0 , "d11_increment":0 }
]
'''
    #{"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 30, "theta8": 0, "theta9": 45, "theta10": 0, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0}
    #{"id_tiro": 2, "theta1": 10, "theta2": 20, "theta4": 0, "theta5": 0, "theta6": 10, "theta8": 60, "theta9": 0, "theta10": 10, "d3_increment": 0, "d7_increment": 30, "d11_increment": 100},
    #{"id_tiro": 3, "theta1": -5, "theta2": 10, "theta4": 0, "theta5": 10, "theta6": 0, "theta8": 0, "theta9": 0, "theta10": 4, "d3_increment":0 , "d7_increment":0 , "d11_increment":0 }

    #{"id_tiro": 1, "theta1": 0, "theta2": 0, "theta4": 0, "theta5": 0, "theta6": 40, "theta8": 0, "theta9": 45, "theta10": 40, "d3_increment": 0, "d7_increment": 0, "d11_increment": 0}


frames_data = json.loads(json_data)

def create_animation(frames_data):
    frames = []
    all_final_positions = []  # Para almacenar todas las posiciones finales progresivamente

    for i, DOF in enumerate(frames_data):
        positions, _, reference_vectors = forward_kinematics(DOF)
        x = positions[:, 0]
        y = -positions[:, 1]  # Invertir el eje Y
        z = positions[:, 2]

        data = [go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                             marker=dict(size=5, color='red'),
                             line=dict(color='blue', width=2))]

        if reference_vectors:
            ref_start, ref_end = reference_vectors[0]
            data.append(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                                     y=[-ref_start[1], -ref_end[1]],  # Invertir el eje Y
                                     z=[ref_start[2], ref_end[2]],
                                     mode='lines',
                                     marker=dict(size=5, color='green'),
                                     line=dict(color='green', width=2)))

        # Añadir punto final con ID del tiro para la posición actual
        final_position = positions[-1]
        all_final_positions.append((final_position[0], -final_position[1], final_position[2], DOF["id_tiro"]))  # Invertir el eje Y

        # Incluir todas las posiciones finales acumuladas
        for pos in all_final_positions:
            data.append(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                                     mode='markers+text', text=[str(pos[3])],
                                     marker=dict(size=5, color='orange')))

        frames.append(go.Frame(data=data, name=str(DOF["id_tiro"])))

    return frames

# Crear la animación
frames = create_animation(frames_data)

# Crear la figura inicial
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'scatter'}]],
                    subplot_titles=('Robotics ARM', 'Posiciones referenciales de tiros'))

# --- Gráfico 3D ---
positions, _, reference_vectors = forward_kinematics(frames_data[0])
x = positions[:, 0]
y = -positions[:, 1]  # Invertir el eje Y
z = positions[:, 2]

trace_3d = go.Scatter3d(x=x, y=y, z=z, mode='lines+markers',
                        marker=dict(size=5, color='red'), line=dict(color='blue', width=2))
fig.add_trace(trace_3d, row=1, col=1)

# Añadir el vector de referencia inicial
if reference_vectors:
    ref_start, ref_end = reference_vectors[0]
    fig.add_trace(go.Scatter3d(x=[ref_start[0], ref_end[0]],
                               y=[-ref_start[1], -ref_end[1]],  # Invertir el eje Y
                               z=[ref_start[2], ref_end[2]],
                               mode='lines',
                               marker=dict(size=5, color='green'),
                               line=dict(color='green', width=2)), row=1, col=1)

# Añadir los puntos finales de todas las posiciones a la figura inicial
all_final_positions = []

for DOF in frames_data:
    positions, _, _ = forward_kinematics(DOF)
    final_position = positions[-1]
    all_final_positions.append((final_position[0], -final_position[1], final_position[2], DOF["id_tiro"]))  # Invertir el eje Y

for pos in all_final_positions:
    fig.add_trace(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]],
                               mode='markers+text', text=[str(pos[3])],
                               marker=dict(size=5, color='orange')), row=1, col=1)

# Añadir los frames a la figura
fig.frames = frames

# Agregar visualización de la frente de carguío (suponiendo que es un plano vertical en X = 7300)
frente_x = 7400  # Posición en el eje X del frente de carguío
frente_y = np.linspace(-2500, 2500, 50)  # Rango en el eje Y
frente_z = np.linspace(0, 5000, 50)  # Rango en el eje Z
frente_y, frente_z = np.meshgrid(frente_y, frente_z)

# Crear la curvatura en la parte superior del frente de carguío
curvatura_superior = 1000 * np.exp(-((frente_y / 2500) ** 2))

# Aplicar la curvatura al frente de carguío
frente_z = np.where(frente_z > 4000, frente_z + curvatura_superior, frente_z)

# Definir un mapa de colores personalizado que se asemeje a la roca
colorscale_roca = [
    [0.0, 'rgb(102, 51, 0)'],
    [0.5, 'rgb(153, 76, 0)'],
    [1.0, 'rgb(204, 102, 0)']
]

fig.add_trace(go.Surface(x=frente_x * np.ones_like(frente_y), y=frente_y, z=frente_z,
                         colorscale=colorscale_roca, opacity=0.3, showscale=False, showlegend=False), row=1, col=1)

# --- Gráfico 2D (Plano YZ) ---
final_positions_2d = np.array([[pos[1], pos[2], pos[3]] for pos in all_final_positions])
trace_2d = go.Scatter(x=final_positions_2d[:, 0], y=final_positions_2d[:, 1],
                      mode='markers+text', text=final_positions_2d[:, 2],
                      textposition='top center',
                      marker=dict(size=8, color='blue'), showlegend=False)
fig.add_trace(trace_2d, row=1, col=2)

# Configurar ejes del gráfico 2D para que coincidan con el gráfico 3D
fig.update_xaxes(title_text='Y', range=[-2500, 2500], row=1, col=2)
fig.update_yaxes(title_text='Z', range=[0, 5000], row=1, col=2)

fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', showticklabels=False, showgrid=True, showline=True),
        yaxis=dict(title='Y', showticklabels=False, showgrid=True, showline=True, autorange='reversed'),  # Invertir el eje Y
        zaxis=dict(title='Z', showticklabels=False, showgrid=True, showline=True),
    ),
    xaxis2=dict(title='Y', showgrid=True, showline=True),
    yaxis2=dict(title='Z', showgrid=True, showline=True),
    title='Representación Brazo UGiT',
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, transition=dict(duration=0))]),
                               dict(label='Pause',
                                    method='animate',
                                    args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])]
)

# Mostrar la figura
fig.show()
